In [2]:
# !pip install loguru
# !pip install nltk

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate
import pandas as pd
from sklearn.model_selection import GridSearchCV
from loguru import logger as lg
import multiprocessing
import nltk
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
import warnings
# Download necessary NLTK corpora
# nltk.download('stopwords')

In [4]:
warnings.filterwarnings("ignore")

In [5]:
NUM_PROCESSORS = (multiprocessing.cpu_count() - 2)
VERBOSE = 0
DEPARTMENTS_QUANTITY_TO_TRAIN = 6
lg.add("finnlp_binary_imbalanced_experiment_1.log", rotation="1024 MB")
lg.info("\n\n####################################### NEW EXECUTION #######################################\n\n")
lg.info(f"Processors (cores) available: {NUM_PROCESSORS}")

2024-02-29 21:40:34.174 | INFO     | __main__:<module>:6 - 

####################################### NEW EXECUTION #######################################


2024-02-29 21:40:34.175 | INFO     | __main__:<module>:7 - Processors (cores) available: 40


In [5]:
brc = pd.read_csv("Banking_Regulation_Corpora_BRC_anonymous.csv" ,encoding="utf-8", low_memory=False, sep=";")

In [6]:
brc.head()

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id,url
0,1,DPT_25,2020-12-23,2128,36560,2020-12-23,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,656,67.0,127.0,COMUNICADO,771984,https://www.bcb.gov.br/estabilidadefinanceira/...
1,1,DPT_25,2020-12-28,2129,36569,2020-12-28,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,2110,156.0,322.0,COMUNICADO,772973,https://www.bcb.gov.br/estabilidadefinanceira/...
2,1,DPT_25,2020-12-22,2130,36554,2020-12-22,BACEN,Divulga comunicado do Grupo de Ação Financeira...,120,16.0,18.0,Comunicamos que o Grupo de Ação Financeira con...,792,76.0,127.0,COMUNICADO,771452,https://www.bcb.gov.br/estabilidadefinanceira/...
3,1,DPT_25,2020-12-23,2131,4880,2020-12-23,BACEN,Dispõe sobre o horário de atendimento ao públi...,222,32.0,34.0,"O Banco Central do Brasil, na forma do art. 9º...",4964,365.0,875.0,RESOLUÇÃO CMN,772051,https://www.bcb.gov.br/estabilidadefinanceira/...
4,1,DPT_25,2020-12-23,2132,1125/2020,2020-12-22,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,165,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,3382,304.0,492.0,PROCESSO ADMINISTRATIVO SANCIONADOR,771966,https://www.bcb.gov.br/estabilidadefinanceira/...


In [7]:
data_irrelevant = brc[brc['class'] == 0]

In [8]:
data_irrelevant.head(3)

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id,url
7846,0,DPT_25,2020-08-31,2169,0301/2020,2020-08-31,BACEN,Caracterizado fornecimento intempestivo ao Ba...,294,36.0,49.0,​DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONAD...,758,81.0,113.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729080,https://www.bcb.gov.br/estabilidadefinanceira/...
7847,0,DPT_25,2020-08-31,2170,36121,2020-08-31,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,7080,407.0,966.0,COMUNICADO,729131,https://www.bcb.gov.br/estabilidadefinanceira/...
7848,0,DPT_25,2020-08-31,2171,36123,2020-08-31,BACEN,Divulga nome de pessoas com intenção de ocupar...,140,19.0,21.0,Divulgamos os nomes de pretendentes a cargos d...,1558,149.0,276.0,COMUNICADO,729151,https://www.bcb.gov.br/estabilidadefinanceira/...


In [9]:
data_relevant = brc[brc['class'] == 1]

In [10]:
data_relevant.count()

class                   7846
department              7846
entry_date              7846
general_id              7846
normative_identifier    7846
publication_date        7846
regulatory_authority    7846
subject                 7846
subject_length          7846
subject_unique_words    7846
subject_words           7846
text                    7846
text_length             7846
text_unique_words       7846
text_words              7846
type                    7846
unique_document_id      7846
url                     7846
dtype: int64

In [11]:
data_relevant = data_relevant[data_relevant.text_words >= 50]
data_relevant.count()

class                   7714
department              7714
entry_date              7714
general_id              7714
normative_identifier    7714
publication_date        7714
regulatory_authority    7714
subject                 7714
subject_length          7714
subject_unique_words    7714
subject_words           7714
text                    7714
text_length             7714
text_unique_words       7714
text_words              7714
type                    7714
unique_document_id      7714
url                     7714
dtype: int64

In [12]:
data_irrelevant.count()

class                   54018
department              54018
entry_date              54018
general_id              54018
normative_identifier    54018
publication_date        54018
regulatory_authority    54018
subject                 54018
subject_length          54018
subject_unique_words    54018
subject_words           54018
text                    54018
text_length             54018
text_unique_words       54018
text_words              54018
type                    54018
unique_document_id      54018
url                     54018
dtype: int64

In [13]:
data_irrelevant = data_irrelevant[data_irrelevant.text_words >= 50]
data_irrelevant.count()

class                   53122
department              53122
entry_date              53122
general_id              53122
normative_identifier    53122
publication_date        53122
regulatory_authority    53122
subject                 53122
subject_length          53122
subject_unique_words    53122
subject_words           53122
text                    53122
text_length             53122
text_unique_words       53122
text_words              53122
type                    53122
unique_document_id      53122
url                     53122
dtype: int64

In [14]:
data_relevant['department'].unique()

array(['DPT_25', 'DPT_3', 'DPT_2', 'DPT_9', 'DPT_5', 'DPT_16', 'DPT_24',
       'DPT_18', 'DPT_12', 'DPT_11', 'DPT_14', 'DPT_4', 'DPT_8', 'DPT_21',
       'DPT_1', 'DPT_15', 'DPT_20', 'DPT_22', 'DPT_10', 'DPT_23', 'DPT_7',
       'DPT_13', 'DPT_17', 'DPT_19', 'DPT_6'], dtype=object)

In [15]:
len(list(data_relevant['department'].unique()))

25

In [16]:
stop_words = set(stopwords.words('portuguese'))
df_stopwords = pd.DataFrame(list(stop_words), columns =['words'])
df_stopwords.head(3)

,words
0,teria
1,aquela
2,um


In [17]:
def remove_special_characters(df,column):
    df.loc[:,column] = df[column].str.lower()
    
    df.loc[:,column] = df[column].str.replace('á','a')
    df.loc[:,column] = df[column].str.replace('é','e')
    df.loc[:,column] = df[column].str.replace('í','i')
    df.loc[:,column] = df[column].str.replace('ó','o')
    df.loc[:,column] = df[column].str.replace('ú','u')

    df.loc[:,column] = df[column].str.replace('â','a')
    df.loc[:,column] = df[column].str.replace('ê','e')
    df.loc[:,column] = df[column].str.replace('î','i')
    df.loc[:,column] = df[column].str.replace('ô','o')
    df.loc[:,column] = df[column].str.replace('û','u')

    df.loc[:,column] = df[column].str.replace('à','a')
    df.loc[:,column] = df[column].str.replace('è','e')
    df.loc[:,column] = df[column].str.replace('ì','i')
    df.loc[:,column] = df[column].str.replace('ò','o')
    df.loc[:,column] = df[column].str.replace('ù','u')

    df.loc[:,column] = df[column].str.replace('ä','a')
    df.loc[:,column] = df[column].str.replace('ë','e')
    df.loc[:,column] = df[column].str.replace('ï','i')
    df.loc[:,column] = df[column].str.replace('ö','o')
    df.loc[:,column] = df[column].str.replace('ü','u')

    df.loc[:,column] = df[column].str.replace('ã','a')
    df.loc[:,column] = df[column].str.replace('ẽ','e')
    df.loc[:,column] = df[column].str.replace('ĩ','i')
    df.loc[:,column] = df[column].str.replace('õ','o')
    df.loc[:,column] = df[column].str.replace('ũ','u')

    df.loc[:,column] = df[column].str.replace('ç','c')
    
    return df

In [18]:
df_stopwords = remove_special_characters(df_stopwords,'words')

In [19]:
df_stopwords.head(3)

,words
0,teria
1,aquela
2,um


In [20]:
stop_words = df_stopwords['words'].tolist()

In [21]:
def clean_text(text, stop_words):
    text = [token for token in text.split(" ")]
    text = [word for word in text if not word in stop_words] # remove stopwords
    text = " ".join(text)
    return text

In [22]:
def cleaning(df,column, stop_words):
    # Remove urls
    df.loc[:,column] = df[column].replace(r'https?:\/\/.*?[\s+]', ' ', regex=True)
    # Remove urls without https?
    df.loc[:,column] = df[column].replace(r'www.*?[\s+]', ' ', regex=True)
    # Remove email address
    df.loc[:,column] = df[column].replace(r'(?P<email_address>[\w\.-]+@[\w\.-]+\.[\w]+)', ' ', regex=True)
    
    df.loc[:,column] = df[column].str.replace('º',' ')
    df.loc[:,column] = df[column].str.replace('ª',' ')

    df.loc[:,column] = df[column].str.replace('1',' ')
    df.loc[:,column] = df[column].str.replace('2',' ')
    df.loc[:,column] = df[column].str.replace('3',' ')
    df.loc[:,column] = df[column].str.replace('4',' ')
    df.loc[:,column] = df[column].str.replace('5',' ')
    df.loc[:,column] = df[column].str.replace('6',' ')
    df.loc[:,column] = df[column].str.replace('7',' ')
    df.loc[:,column] = df[column].str.replace('8',' ')
    df.loc[:,column] = df[column].str.replace('9',' ')
    df.loc[:,column] = df[column].str.replace('0',' ')

    df.loc[:,column] = df[column].str.replace('/',' ')

    df.loc[:,column] = df[column].str.replace('\r',' ')
    df.loc[:,column] = df[column].str.replace('\n',' ')
    df.loc[:,column] = df[column].str.replace('\t',' ')
    df.loc[:,column] = df[column].str.replace('\\',' ')

    df.loc[:,column] = df[column].str.replace('-',' ')

    df.loc[:,column] = df[column].str.replace('–',' ')
    df.loc[:,column] = df[column].str.replace('“',' ')
    df.loc[:,column] = df[column].str.replace('”',' ')
    df.loc[:,column] = df[column].str.replace('’',' ')
    df.loc[:,column] = df[column].str.replace('_',' ')
    df.loc[:,column] = df[column].str.replace('.',' ')
    df.loc[:,column] = df[column].str.replace(',',' ')
    df.loc[:,column] = df[column].str.replace('|',' ')
    df.loc[:,column] = df[column].str.replace('=',' ')
    df.loc[:,column] = df[column].str.replace('@',' ')
    df.loc[:,column] = df[column].str.replace('$',' ')

    df.loc[:,column] = df[column].str.replace('°',' ')
    df.loc[:,column] = df[column].str.replace('§',' ')
    df.loc[:,column] = df[column].str.replace('•',' ')
    df.loc[:,column] = df[column].str.replace('▪',' ')

    df.loc[:,column] = df[column].str.replace('%',' ')
    df.loc[:,column] = df[column].str.replace('&',' ')
    df.loc[:,column] = df[column].str.replace('*',' ')
    df.loc[:,column] = df[column].str.replace('+',' ')
    df.loc[:,column] = df[column].str.replace(':',' ')
    df.loc[:,column] = df[column].str.replace(';',' ')

    df.loc[:,column] = df[column].str.replace('!',' ')
    df.loc[:,column] = df[column].str.replace('?',' ')
    df.loc[:,column] = df[column].str.replace('#',' ')
    df.loc[:,column] = df[column].str.replace('\'',' ')
    df.loc[:,column] = df[column].str.replace('"',' ')

    df.loc[:,column] = df[column].str.replace('<',' ')
    df.loc[:,column] = df[column].str.replace('>',' ')

    df.loc[:,column] = df[column].str.replace('(',' ')
    df.loc[:,column] = df[column].str.replace(')',' ')

    df.loc[:,column] = df[column].str.replace('{',' ')
    df.loc[:,column] = df[column].str.replace('}',' ')

    df.loc[:,column] = df[column].str.replace('[',' ')
    df.loc[:,column] = df[column].str.replace(']',' ')

    # Remove the same character repeated more than twice
    df.loc[:,column] = df[column].replace(r'([a-z])\1{2,}', ' ', regex=True)
    
    # To replace more than one white space by only one white space
    df.loc[:,column] = df[column].replace(r'\s+', ' ', regex=True)

    # Remove stopwords
    df.loc[:,column] = df[column].apply(lambda x: clean_text(x, stop_words))

    return df

In [23]:
def remove_duplicates(sentence):
	words = sentence.split(" ")
	result = []
	for word in words:
		if word not in result:
			result.append(word)
	return result

<h3>Selecting the Departments</h3>

In [24]:
boards = data_relevant['department']
# Getting the 6 most populated boards
boards_list = list(boards.unique())[:DEPARTMENTS_QUANTITY_TO_TRAIN]
lg.info(f"boards_list: {boards_list}")

2024-02-29 21:32:15.554 | INFO     | __main__:<module>:4 - boards_list: ['DPT_25', 'DPT_3', 'DPT_2', 'DPT_9', 'DPT_5', 'DPT_16']


<h3>Parameters</h3>

In [25]:
from sklearn.metrics import make_scorer
import sklearn.metrics as sk

METRICS = {
           'accuracy': make_scorer(sk.accuracy_score),
           'precision': make_scorer(sk.precision_score, average = 'weighted', zero_division=0),
           'recall': make_scorer(sk.recall_score, average = 'weighted', zero_division=0),
           'f1': make_scorer(sk.f1_score, average = 'binary', zero_division=0),
           'f1_macro': make_scorer(sk.f1_score, average = 'macro', zero_division=0),
           'f1_weighted': make_scorer(sk.f1_score, average = 'weighted', zero_division=0)
           }

CROSS_VALIDATION = 10
RAND_STATE = 42
REFIT_VAL = 'f1'

cls = dict({
        'random_forest': {
            'estimator': RandomForestClassifier(random_state=RAND_STATE, n_jobs = NUM_PROCESSORS),
            'parameters':{'n_estimators': [100, 400, 1000], 'max_depth': [10,30, 100], 'criterion':['entropy', 'log_loss', 'gini']}
            },
        'svm': {
            'estimator': SVC(random_state=RAND_STATE),
            'parameters':{'C': [0.025, 0.08, 0.1, 0.5, 0.8, 1.0, 2.0, 10.0, 100.0, 500.0, 1000.0], 'kernel': ['linear','poly','rbf','sigmoid']}
            },
        'xgboost':{
            'estimator': XGBClassifier(random_state=RAND_STATE, num_class=2, verbosity = 0, silent=True, n_jobs=NUM_PROCESSORS),
            'parameters':{'objective':['reg:squarederror','binary:logistic','multi:softmax','binary:hinge'],'n_estimators': [100,1000],'max_depth': [10,30],'learning_rate':[0.01,0.5]}
        },
        'nb':{
                'estimator': MultinomialNB(),
                'parameters':{'alpha':[1, 0.1, 0.01, 0.001, 0.0001, 0.00001],'force_alpha': [True, False],'fit_prior': [True, False]}
            }
        
    })

In [26]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

def training_each_inductor_holdout(classifiers, board, X, y):
    for key in list(classifiers.keys()):
        lg.info(f"\n\n####### {key} INDUCTOR EXECUTING ####### - {board}\n\n")

        lg.debug(f"[+] classifiers[key]['estimator']: {classifiers[key]['estimator']} - {key} - {board}")
        lg.debug(f"[+] classifiers[key]['parameters']: {classifiers[key]['parameters']} - {key} - {board}")

        search = GridSearchCV(estimator=classifiers[key]['estimator'],
                            param_grid=classifiers[key]['parameters'],
                            cv = CROSS_VALIDATION,
                            scoring = METRICS,
                            refit = REFIT_VAL,
                            n_jobs = NUM_PROCESSORS,
                            verbose = VERBOSE)

        lg.debug(f"[+] Start GridSearchCV - {key} - {board}")
        search.fit(X, y)
        lg.debug(f"[+] End GridSearchCV - {key} - {board}")

        lg.info(f"search.best_estimator_: {search.best_estimator_} \n")

        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=RAND_STATE, stratify=y)

        clf = search.best_estimator_.fit(x_train, y_train)
        
        y_pred = clf.predict(x_test)

        lg.success(f"\n\n####### METRICS {key} - {board} #######\n\n")

        lg.success(f"{key} - {board} - test_precision: {metrics.precision_score(y_test, y_pred)}")

        lg.success(f"{key} - {board} - test_recall: {metrics.recall_score(y_test, y_pred)}")

        lg.success(f"{key} - {board} - test_f1_binary: {metrics.f1_score(y_test, y_pred)}")

        lg.success(f"{key} - {board} - test_f1_macro: {metrics.f1_score(y_test, y_pred, average='macro')}")

<h3>The Training and Evaluation</h3>

In [ ]:
for board in boards_list:
    lg.info(f"\n\n####### {board} EXECUTING #######\n\n")

    total_relevant_documents_of_the_board = data_relevant[data_relevant.department == board].shape[0]
    
    ######################################################################
    ### TO TEST IF THE PIPELINE IS WORKING - COMMENT AFTER TEST IS DONE###

    # total_relevant_documents_of_the_board = int(data_relevant[data_relevant.department == board].shape[0]/10)

    ### TO TEST IF THE PIPELINE IS WORKING - COMMENT AFTER TEST IS DONE###
    ######################################################################

    lg.info(f"total_relevant_documents_of_the_board: {total_relevant_documents_of_the_board} - {board}")
    lg.info(f"Total irrelevant documents of the department: {data_irrelevant[data_irrelevant.department == board].shape[0]}")

    data_relevant_board = data_relevant[data_relevant.department == board]
    lg.info(f"data_relevant_board: {data_relevant_board.shape} - {board}")

    ####################################################################################################
    ############## TO TRAIN WITH A BALANCED DATASET (UNDERSAMPLING) OR IMBALANCED DATASET ##############
    # The total documents of the relevant class (total_relevant_documents_of_the_board) is use to get the first documents in the irrelevant class.

    # data_irrelevant_board = data_irrelevant[data_irrelevant.department == board][:total_relevant_documents_of_the_board]
    data_irrelevant_board = data_irrelevant[data_irrelevant.department == board]

    ############## TO TRAIN WITH A BALANCED DATASET (UNDERSAMPLING) OR IMBALANCED DATASET ##############
    ####################################################################################################

    ######################################################################
    ### TO TEST IF THE PIPELINE IS WORKING - COMMENT AFTER TEST IS DONE###

    # data_relevant_board = data_relevant[data_relevant.department == board][:total_relevant_documents_of_the_board]
    
    ### TO TEST IF THE PIPELINE IS WORKING - COMMENT AFTER TEST IS DONE###
    ######################################################################

    # data_irrelevant_board = data_irrelevant[data_irrelevant.department == board]
    lg.info(f"data_irrelevant_board: {data_irrelevant_board.shape} - {board}")

    df_gather_datasets = []
    df_gather_datasets.append(data_relevant_board)
    df_gather_datasets.append(data_irrelevant_board)
    # Merging all dataframes in the list as one dataframe.
    df_final_dataset = pd.concat(df_gather_datasets)
    
    lg.info(f"df_final_dataset: {df_final_dataset.shape} - {board}")

    # Cleaning the text and title
    lg.debug("[+] Start cleaning text and title")
    df_final_dataset = remove_special_characters(df_final_dataset, 'text')
    df_final_dataset = remove_special_characters(df_final_dataset, 'subject')
    df_final_dataset = cleaning(df_final_dataset, 'text', stop_words)
    df_final_dataset = cleaning(df_final_dataset, 'subject', stop_words)
    lg.debug("[+] End cleaning text and title")

    # To get only phrases with at lest three words after the preprocessing.
    df_final_dataset = df_final_dataset[df_final_dataset['text'].str.len() >= 3]

    data_x = df_final_dataset.text
    lg.debug("[+] Start TF-IDF")

    feature_extraction = TfidfVectorizer()
    data_x = feature_extraction.fit_transform(data_x)
    lg.debug("[+] End TF-IDF")

    data_y = df_final_dataset['class']

    training_each_inductor_holdout(cls, board, data_x, data_y)

    lg.success("\n\n[+][+][+] SUCCESS! [+][+][+]\n\n")

<h2>The output result is in the next cell. It was copied from the log file.</h2>

In [ ]:
# 2024-01-26 09:02:28.047 | INFO     | __main__:<module>:5 - 

# ####################################### NEW EXECUTION #######################################


# 2024-01-26 09:02:28.048 | INFO     | __main__:<module>:6 - Processors (cores) available: 38
# 2024-01-26 09:02:38.261 | INFO     | __main__:<module>:4 - boards_list: ['DPT_25', 'DPT_3', 'DPT_2', 'DPT_9', 'DPT_5', 'DPT_16']
# 2024-01-26 09:02:38.294 | INFO     | __main__:<module>:2 - 

# ####### DPT_25 EXECUTING #######


# 2024-01-26 09:02:38.296 | INFO     | __main__:<module>:14 - total_relevant_documents_of_the_board: 819 - DPT_25
# 2024-01-26 09:02:38.302 | INFO     | __main__:<module>:15 - Total irrelevant documents of the department: 4051
# 2024-01-26 09:02:38.304 | INFO     | __main__:<module>:18 - data_relevant_board: (819, 18) - DPT_25
# 2024-01-26 09:02:38.308 | INFO     | __main__:<module>:39 - data_irrelevant_board: (4051, 18) - DPT_25
# 2024-01-26 09:02:38.312 | INFO     | __main__:<module>:47 - df_final_dataset: (4870, 18) - DPT_25
# 2024-01-26 09:02:38.313 | DEBUG    | __main__:<module>:50 - [+] Start cleaning text and title
# 2024-01-26 09:03:15.134 | DEBUG    | __main__:<module>:55 - [+] End cleaning text and title
# 2024-01-26 09:03:15.141 | DEBUG    | __main__:<module>:61 - [+] Start TF-IDF
# 2024-01-26 09:03:19.648 | DEBUG    | __main__:<module>:65 - [+] End TF-IDF
# 2024-01-26 09:03:19.650 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### random_forest INDUCTOR EXECUTING ####### - DPT_25


# 2024-01-26 09:03:19.652 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: RandomForestClassifier(n_jobs=38, random_state=42) - random_forest - DPT_25
# 2024-01-26 09:03:19.652 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'n_estimators': [100, 400, 1000], 'max_depth': [10, 30, 100], 'criterion': ['entropy', 'log_loss', 'gini']} - random_forest - DPT_25
# 2024-01-26 09:03:19.653 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - random_forest - DPT_25
# 2024-01-26 09:07:37.675 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - random_forest - DPT_25
# 2024-01-26 09:07:37.678 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: RandomForestClassifier(max_depth=100, n_jobs=38, random_state=42) 

# 2024-01-26 09:07:38.305 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS random_forest - DPT_25 #######


# 2024-01-26 09:07:38.312 | SUCCESS  | __main__:training_each_inductor_holdout:33 - random_forest - DPT_25 - test_precision: 0.9469387755102041
# 2024-01-26 09:07:38.315 | SUCCESS  | __main__:training_each_inductor_holdout:35 - random_forest - DPT_25 - test_recall: 0.7073170731707317
# 2024-01-26 09:07:38.319 | SUCCESS  | __main__:training_each_inductor_holdout:37 - random_forest - DPT_25 - test_f1_binary: 0.8097731239092496
# 2024-01-26 09:07:38.322 | SUCCESS  | __main__:training_each_inductor_holdout:39 - random_forest - DPT_25 - test_f1_macro: 0.8884857193425273
# 2024-01-26 09:07:38.322 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### svm INDUCTOR EXECUTING ####### - DPT_25


# 2024-01-26 09:07:38.323 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: SVC(random_state=42) - svm - DPT_25
# 2024-01-26 09:07:38.324 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'C': [0.025, 0.08, 0.1, 0.5, 0.8, 1.0, 2.0, 10.0, 100.0, 500.0, 1000.0], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} - svm - DPT_25
# 2024-01-26 09:07:38.325 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - svm - DPT_25
# 2024-01-26 09:14:43.895 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - svm - DPT_25
# 2024-01-26 09:14:43.897 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: SVC(C=10.0, kernel='linear', random_state=42) 

# 2024-01-26 09:14:55.266 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS svm - DPT_25 #######


# 2024-01-26 09:14:55.271 | SUCCESS  | __main__:training_each_inductor_holdout:33 - svm - DPT_25 - test_precision: 0.8945578231292517
# 2024-01-26 09:14:55.274 | SUCCESS  | __main__:training_each_inductor_holdout:35 - svm - DPT_25 - test_recall: 0.801829268292683
# 2024-01-26 09:14:55.277 | SUCCESS  | __main__:training_each_inductor_holdout:37 - svm - DPT_25 - test_f1_binary: 0.8456591639871384
# 2024-01-26 09:14:55.280 | SUCCESS  | __main__:training_each_inductor_holdout:39 - svm - DPT_25 - test_f1_macro: 0.9081686168133615
# 2024-01-26 09:14:55.281 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### xgboost INDUCTOR EXECUTING ####### - DPT_25


# 2024-01-26 09:14:55.283 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=None, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=None, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=None, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) - xgboost - DPT_25
# 2024-01-26 09:14:55.284 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'objective': ['reg:squarederror', 'binary:logistic', 'multi:softmax', 'binary:hinge'], 'n_estimators': [100, 1000], 'max_depth': [10, 30], 'learning_rate': [0.01, 0.5]} - xgboost - DPT_25
# 2024-01-26 09:14:55.284 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - xgboost - DPT_25
# 2024-01-26 15:37:26.786 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - xgboost - DPT_25
# 2024-01-26 15:37:26.790 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=0.5, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=10, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=1000, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) 

# 2024-01-26 15:38:29.506 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS xgboost - DPT_25 #######


# 2024-01-26 15:38:29.511 | SUCCESS  | __main__:training_each_inductor_holdout:33 - xgboost - DPT_25 - test_precision: 0.92578125
# 2024-01-26 15:38:29.514 | SUCCESS  | __main__:training_each_inductor_holdout:35 - xgboost - DPT_25 - test_recall: 0.7225609756097561
# 2024-01-26 15:38:29.518 | SUCCESS  | __main__:training_each_inductor_holdout:37 - xgboost - DPT_25 - test_f1_binary: 0.8116438356164384
# 2024-01-26 15:38:29.521 | SUCCESS  | __main__:training_each_inductor_holdout:39 - xgboost - DPT_25 - test_f1_macro: 0.8892156376149825
# 2024-01-26 15:38:29.522 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### nb INDUCTOR EXECUTING ####### - DPT_25


# 2024-01-26 15:38:29.523 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: MultinomialNB() - nb - DPT_25
# 2024-01-26 15:38:29.523 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05], 'force_alpha': [True, False], 'fit_prior': [True, False]} - nb - DPT_25
# 2024-01-26 15:38:29.524 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - nb - DPT_25
# 2024-01-26 15:38:31.907 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - nb - DPT_25
# 2024-01-26 15:38:31.909 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: MultinomialNB(alpha=0.1, force_alpha=True) 

# 2024-01-26 15:38:31.944 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS nb - DPT_25 #######


# 2024-01-26 15:38:31.948 | SUCCESS  | __main__:training_each_inductor_holdout:33 - nb - DPT_25 - test_precision: 0.941908713692946
# 2024-01-26 15:38:31.951 | SUCCESS  | __main__:training_each_inductor_holdout:35 - nb - DPT_25 - test_recall: 0.6920731707317073
# 2024-01-26 15:38:31.955 | SUCCESS  | __main__:training_each_inductor_holdout:37 - nb - DPT_25 - test_f1_binary: 0.797891036906854
# 2024-01-26 15:38:31.958 | SUCCESS  | __main__:training_each_inductor_holdout:39 - nb - DPT_25 - test_f1_macro: 0.881662681062384
# 2024-01-26 15:38:31.960 | SUCCESS  | __main__:<module>:72 - 

# [+][+][+] SUCCESS! [+][+][+]


# 2024-01-26 15:38:31.961 | INFO     | __main__:<module>:2 - 

# ####### DPT_3 EXECUTING #######


# 2024-01-26 15:38:31.965 | INFO     | __main__:<module>:14 - total_relevant_documents_of_the_board: 688 - DPT_3
# 2024-01-26 15:38:31.973 | INFO     | __main__:<module>:15 - Total irrelevant documents of the department: 1790
# 2024-01-26 15:38:31.975 | INFO     | __main__:<module>:18 - data_relevant_board: (688, 18) - DPT_3
# 2024-01-26 15:38:31.979 | INFO     | __main__:<module>:39 - data_irrelevant_board: (1790, 18) - DPT_3
# 2024-01-26 15:38:31.998 | INFO     | __main__:<module>:47 - df_final_dataset: (2478, 18) - DPT_3
# 2024-01-26 15:38:31.999 | DEBUG    | __main__:<module>:50 - [+] Start cleaning text and title
# 2024-01-26 15:39:23.197 | DEBUG    | __main__:<module>:55 - [+] End cleaning text and title
# 2024-01-26 15:39:23.202 | DEBUG    | __main__:<module>:61 - [+] Start TF-IDF
# 2024-01-26 15:39:28.839 | DEBUG    | __main__:<module>:65 - [+] End TF-IDF
# 2024-01-26 15:39:28.841 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### random_forest INDUCTOR EXECUTING ####### - DPT_3


# 2024-01-26 15:39:28.842 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: RandomForestClassifier(n_jobs=38, random_state=42) - random_forest - DPT_3
# 2024-01-26 15:39:28.843 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'n_estimators': [100, 400, 1000], 'max_depth': [10, 30, 100], 'criterion': ['entropy', 'log_loss', 'gini']} - random_forest - DPT_3
# 2024-01-26 15:39:28.843 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - random_forest - DPT_3
# 2024-01-26 15:43:57.403 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - random_forest - DPT_3
# 2024-01-26 15:43:57.406 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: RandomForestClassifier(max_depth=100, n_estimators=400, n_jobs=38,
#                        random_state=42) 

# 2024-01-26 15:43:59.318 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS random_forest - DPT_3 #######


# 2024-01-26 15:43:59.327 | SUCCESS  | __main__:training_each_inductor_holdout:33 - random_forest - DPT_3 - test_precision: 0.7732558139534884
# 2024-01-26 15:43:59.330 | SUCCESS  | __main__:training_each_inductor_holdout:35 - random_forest - DPT_3 - test_recall: 0.48363636363636364
# 2024-01-26 15:43:59.334 | SUCCESS  | __main__:training_each_inductor_holdout:37 - random_forest - DPT_3 - test_f1_binary: 0.5950782997762863
# 2024-01-26 15:43:59.337 | SUCCESS  | __main__:training_each_inductor_holdout:39 - random_forest - DPT_3 - test_f1_macro: 0.7386582129980976
# 2024-01-26 15:43:59.337 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### svm INDUCTOR EXECUTING ####### - DPT_3


# 2024-01-26 15:43:59.338 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: SVC(random_state=42) - svm - DPT_3
# 2024-01-26 15:43:59.339 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'C': [0.025, 0.08, 0.1, 0.5, 0.8, 1.0, 2.0, 10.0, 100.0, 500.0, 1000.0], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} - svm - DPT_3
# 2024-01-26 15:43:59.340 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - svm - DPT_3
# 2024-01-26 15:47:05.372 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - svm - DPT_3
# 2024-01-26 15:47:05.374 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: SVC(C=100.0, kernel='linear', random_state=42) 

# 2024-01-26 15:47:11.242 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS svm - DPT_3 #######


# 2024-01-26 15:47:11.246 | SUCCESS  | __main__:training_each_inductor_holdout:33 - svm - DPT_3 - test_precision: 0.6462093862815884
# 2024-01-26 15:47:11.249 | SUCCESS  | __main__:training_each_inductor_holdout:35 - svm - DPT_3 - test_recall: 0.6509090909090909
# 2024-01-26 15:47:11.252 | SUCCESS  | __main__:training_each_inductor_holdout:37 - svm - DPT_3 - test_f1_binary: 0.6485507246376813
# 2024-01-26 15:47:11.255 | SUCCESS  | __main__:training_each_inductor_holdout:39 - svm - DPT_3 - test_f1_macro: 0.7565379321512429
# 2024-01-26 15:47:11.256 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### xgboost INDUCTOR EXECUTING ####### - DPT_3


# 2024-01-26 15:47:11.257 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=None, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=None, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=None, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) - xgboost - DPT_3
# 2024-01-26 15:47:11.258 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'objective': ['reg:squarederror', 'binary:logistic', 'multi:softmax', 'binary:hinge'], 'n_estimators': [100, 1000], 'max_depth': [10, 30], 'learning_rate': [0.01, 0.5]} - xgboost - DPT_3
# 2024-01-26 15:47:11.259 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - xgboost - DPT_3
# 2024-01-26 23:51:08.899 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - xgboost - DPT_3
# 2024-01-26 23:51:08.903 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=0.5, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=10, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=1000, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) 

# 2024-01-26 23:52:31.508 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS xgboost - DPT_3 #######


# 2024-01-26 23:52:31.513 | SUCCESS  | __main__:training_each_inductor_holdout:33 - xgboost - DPT_3 - test_precision: 0.7350427350427351
# 2024-01-26 23:52:31.516 | SUCCESS  | __main__:training_each_inductor_holdout:35 - xgboost - DPT_3 - test_recall: 0.6254545454545455
# 2024-01-26 23:52:31.520 | SUCCESS  | __main__:training_each_inductor_holdout:37 - xgboost - DPT_3 - test_f1_binary: 0.6758349705304519
# 2024-01-26 23:52:31.523 | SUCCESS  | __main__:training_each_inductor_holdout:39 - xgboost - DPT_3 - test_f1_macro: 0.7819852818753954
# 2024-01-26 23:52:31.524 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### nb INDUCTOR EXECUTING ####### - DPT_3


# 2024-01-26 23:52:31.525 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: MultinomialNB() - nb - DPT_3
# 2024-01-26 23:52:31.525 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05], 'force_alpha': [True, False], 'fit_prior': [True, False]} - nb - DPT_3
# 2024-01-26 23:52:31.526 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - nb - DPT_3
# 2024-01-26 23:52:34.039 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - nb - DPT_3
# 2024-01-26 23:52:34.041 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: MultinomialNB(alpha=0.001, fit_prior=False, force_alpha=True) 

# 2024-01-26 23:52:34.071 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS nb - DPT_3 #######


# 2024-01-26 23:52:34.075 | SUCCESS  | __main__:training_each_inductor_holdout:33 - nb - DPT_3 - test_precision: 0.5025380710659898
# 2024-01-26 23:52:34.079 | SUCCESS  | __main__:training_each_inductor_holdout:35 - nb - DPT_3 - test_recall: 0.72
# 2024-01-26 23:52:34.082 | SUCCESS  | __main__:training_each_inductor_holdout:37 - nb - DPT_3 - test_f1_binary: 0.5919282511210763
# 2024-01-26 23:52:34.085 | SUCCESS  | __main__:training_each_inductor_holdout:39 - nb - DPT_3 - test_f1_macro: 0.6921618441917168
# 2024-01-26 23:52:34.086 | SUCCESS  | __main__:<module>:72 - 

# [+][+][+] SUCCESS! [+][+][+]


# 2024-01-26 23:52:34.087 | INFO     | __main__:<module>:2 - 

# ####### DPT_2 EXECUTING #######


# 2024-01-26 23:52:34.092 | INFO     | __main__:<module>:14 - total_relevant_documents_of_the_board: 683 - DPT_2
# 2024-01-26 23:52:34.105 | INFO     | __main__:<module>:15 - Total irrelevant documents of the department: 5234
# 2024-01-26 23:52:34.108 | INFO     | __main__:<module>:18 - data_relevant_board: (683, 18) - DPT_2
# 2024-01-26 23:52:34.113 | INFO     | __main__:<module>:39 - data_irrelevant_board: (5234, 18) - DPT_2
# 2024-01-26 23:52:34.128 | INFO     | __main__:<module>:47 - df_final_dataset: (5917, 18) - DPT_2
# 2024-01-26 23:52:34.129 | DEBUG    | __main__:<module>:50 - [+] Start cleaning text and title
# 2024-01-26 23:53:38.484 | DEBUG    | __main__:<module>:55 - [+] End cleaning text and title
# 2024-01-26 23:53:38.492 | DEBUG    | __main__:<module>:61 - [+] Start TF-IDF
# 2024-01-26 23:53:45.606 | DEBUG    | __main__:<module>:65 - [+] End TF-IDF
# 2024-01-26 23:53:45.608 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### random_forest INDUCTOR EXECUTING ####### - DPT_2


# 2024-01-26 23:53:45.610 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: RandomForestClassifier(n_jobs=38, random_state=42) - random_forest - DPT_2
# 2024-01-26 23:53:45.611 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'n_estimators': [100, 400, 1000], 'max_depth': [10, 30, 100], 'criterion': ['entropy', 'log_loss', 'gini']} - random_forest - DPT_2
# 2024-01-26 23:53:45.611 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - random_forest - DPT_2
# 2024-01-27 00:07:22.746 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - random_forest - DPT_2
# 2024-01-27 00:07:22.749 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: RandomForestClassifier(max_depth=100, n_jobs=38, random_state=42) 

# 2024-01-27 00:07:23.472 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS random_forest - DPT_2 #######


# 2024-01-27 00:07:23.479 | SUCCESS  | __main__:training_each_inductor_holdout:33 - random_forest - DPT_2 - test_precision: 0.680672268907563
# 2024-01-27 00:07:23.483 | SUCCESS  | __main__:training_each_inductor_holdout:35 - random_forest - DPT_2 - test_recall: 0.2967032967032967
# 2024-01-27 00:07:23.486 | SUCCESS  | __main__:training_each_inductor_holdout:37 - random_forest - DPT_2 - test_f1_binary: 0.413265306122449
# 2024-01-27 00:07:23.489 | SUCCESS  | __main__:training_each_inductor_holdout:39 - random_forest - DPT_2 - test_f1_macro: 0.6801471625038777
# 2024-01-27 00:07:23.490 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### svm INDUCTOR EXECUTING ####### - DPT_2


# 2024-01-27 00:07:23.491 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: SVC(random_state=42) - svm - DPT_2
# 2024-01-27 00:07:23.492 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'C': [0.025, 0.08, 0.1, 0.5, 0.8, 1.0, 2.0, 10.0, 100.0, 500.0, 1000.0], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} - svm - DPT_2
# 2024-01-27 00:07:23.492 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - svm - DPT_2
# 2024-01-27 00:16:37.348 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - svm - DPT_2
# 2024-01-27 00:16:37.350 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: SVC(C=2.0, kernel='linear', random_state=42) 

# 2024-01-27 00:16:53.857 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS svm - DPT_2 #######


# 2024-01-27 00:16:53.862 | SUCCESS  | __main__:training_each_inductor_holdout:33 - svm - DPT_2 - test_precision: 0.7081339712918661
# 2024-01-27 00:16:53.865 | SUCCESS  | __main__:training_each_inductor_holdout:35 - svm - DPT_2 - test_recall: 0.5421245421245421
# 2024-01-27 00:16:53.868 | SUCCESS  | __main__:training_each_inductor_holdout:37 - svm - DPT_2 - test_f1_binary: 0.6141078838174273
# 2024-01-27 00:16:53.871 | SUCCESS  | __main__:training_each_inductor_holdout:39 - svm - DPT_2 - test_f1_macro: 0.7851818817017523
# 2024-01-27 00:16:53.872 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### xgboost INDUCTOR EXECUTING ####### - DPT_2


# 2024-01-27 00:16:53.873 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=None, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=None, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=None, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) - xgboost - DPT_2
# 2024-01-27 00:16:53.874 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'objective': ['reg:squarederror', 'binary:logistic', 'multi:softmax', 'binary:hinge'], 'n_estimators': [100, 1000], 'max_depth': [10, 30], 'learning_rate': [0.01, 0.5]} - xgboost - DPT_2
# 2024-01-27 00:16:53.875 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - xgboost - DPT_2
# 2024-01-27 10:51:06.928 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - xgboost - DPT_2
# 2024-01-27 10:51:06.933 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=0.01, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=30, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=100, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) 

# 2024-01-27 10:52:54.176 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS xgboost - DPT_2 #######


# 2024-01-27 10:52:54.181 | SUCCESS  | __main__:training_each_inductor_holdout:33 - xgboost - DPT_2 - test_precision: 0.6108597285067874
# 2024-01-27 10:52:54.185 | SUCCESS  | __main__:training_each_inductor_holdout:35 - xgboost - DPT_2 - test_recall: 0.4945054945054945
# 2024-01-27 10:52:54.188 | SUCCESS  | __main__:training_each_inductor_holdout:37 - xgboost - DPT_2 - test_f1_binary: 0.5465587044534413
# 2024-01-27 10:52:54.192 | SUCCESS  | __main__:training_each_inductor_holdout:39 - xgboost - DPT_2 - test_f1_macro: 0.7468642578870981
# 2024-01-27 10:52:54.193 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### nb INDUCTOR EXECUTING ####### - DPT_2


# 2024-01-27 10:52:54.194 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: MultinomialNB() - nb - DPT_2
# 2024-01-27 10:52:54.194 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05], 'force_alpha': [True, False], 'fit_prior': [True, False]} - nb - DPT_2
# 2024-01-27 10:52:54.195 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - nb - DPT_2
# 2024-01-27 10:52:56.896 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - nb - DPT_2
# 2024-01-27 10:52:56.899 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: MultinomialNB(alpha=0.001, force_alpha=True) 

# 2024-01-27 10:52:56.938 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS nb - DPT_2 #######


# 2024-01-27 10:52:56.943 | SUCCESS  | __main__:training_each_inductor_holdout:33 - nb - DPT_2 - test_precision: 0.5483870967741935
# 2024-01-27 10:52:56.947 | SUCCESS  | __main__:training_each_inductor_holdout:35 - nb - DPT_2 - test_recall: 0.684981684981685
# 2024-01-27 10:52:56.950 | SUCCESS  | __main__:training_each_inductor_holdout:37 - nb - DPT_2 - test_f1_binary: 0.6091205211726384
# 2024-01-27 10:52:56.954 | SUCCESS  | __main__:training_each_inductor_holdout:39 - nb - DPT_2 - test_f1_macro: 0.7754340469940861
# 2024-01-27 10:52:56.955 | SUCCESS  | __main__:<module>:72 - 

# [+][+][+] SUCCESS! [+][+][+]


# 2024-01-27 10:52:56.956 | INFO     | __main__:<module>:2 - 

# ####### DPT_9 EXECUTING #######


# 2024-01-27 10:52:56.960 | INFO     | __main__:<module>:14 - total_relevant_documents_of_the_board: 600 - DPT_9
# 2024-01-27 10:52:56.972 | INFO     | __main__:<module>:15 - Total irrelevant documents of the department: 3619
# 2024-01-27 10:52:56.974 | INFO     | __main__:<module>:18 - data_relevant_board: (600, 18) - DPT_9
# 2024-01-27 10:52:56.980 | INFO     | __main__:<module>:39 - data_irrelevant_board: (3619, 18) - DPT_9
# 2024-01-27 10:52:57.006 | INFO     | __main__:<module>:47 - df_final_dataset: (4219, 18) - DPT_9
# 2024-01-27 10:52:57.007 | DEBUG    | __main__:<module>:50 - [+] Start cleaning text and title
# 2024-01-27 10:53:33.395 | DEBUG    | __main__:<module>:55 - [+] End cleaning text and title
# 2024-01-27 10:53:33.402 | DEBUG    | __main__:<module>:61 - [+] Start TF-IDF
# 2024-01-27 10:53:37.710 | DEBUG    | __main__:<module>:65 - [+] End TF-IDF
# 2024-01-27 10:53:37.712 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### random_forest INDUCTOR EXECUTING ####### - DPT_9


# 2024-01-27 10:53:37.713 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: RandomForestClassifier(n_jobs=38, random_state=42) - random_forest - DPT_9
# 2024-01-27 10:53:37.715 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'n_estimators': [100, 400, 1000], 'max_depth': [10, 30, 100], 'criterion': ['entropy', 'log_loss', 'gini']} - random_forest - DPT_9
# 2024-01-27 10:53:37.716 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - random_forest - DPT_9
# 2024-01-27 11:13:47.310 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - random_forest - DPT_9
# 2024-01-27 11:13:47.312 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: RandomForestClassifier(max_depth=100, n_jobs=38, random_state=42) 

# 2024-01-27 11:13:48.895 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS random_forest - DPT_9 #######


# 2024-01-27 11:13:48.901 | SUCCESS  | __main__:training_each_inductor_holdout:33 - random_forest - DPT_9 - test_precision: 0.5533333333333333
# 2024-01-27 11:13:48.905 | SUCCESS  | __main__:training_each_inductor_holdout:35 - random_forest - DPT_9 - test_recall: 0.3458333333333333
# 2024-01-27 11:13:48.908 | SUCCESS  | __main__:training_each_inductor_holdout:37 - random_forest - DPT_9 - test_f1_binary: 0.4256410256410257
# 2024-01-27 11:13:48.911 | SUCCESS  | __main__:training_each_inductor_holdout:39 - random_forest - DPT_9 - test_f1_macro: 0.6753121404159583
# 2024-01-27 11:13:48.911 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### svm INDUCTOR EXECUTING ####### - DPT_9


# 2024-01-27 11:13:48.912 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: SVC(random_state=42) - svm - DPT_9
# 2024-01-27 11:13:48.913 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'C': [0.025, 0.08, 0.1, 0.5, 0.8, 1.0, 2.0, 10.0, 100.0, 500.0, 1000.0], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} - svm - DPT_9
# 2024-01-27 11:13:48.914 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - svm - DPT_9
# 2024-01-27 11:17:38.805 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - svm - DPT_9
# 2024-01-27 11:17:38.807 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: SVC(C=2.0, kernel='linear', random_state=42) 

# 2024-01-27 11:17:46.594 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS svm - DPT_9 #######


# 2024-01-27 11:17:46.599 | SUCCESS  | __main__:training_each_inductor_holdout:33 - svm - DPT_9 - test_precision: 0.6039603960396039
# 2024-01-27 11:17:46.602 | SUCCESS  | __main__:training_each_inductor_holdout:35 - svm - DPT_9 - test_recall: 0.5083333333333333
# 2024-01-27 11:17:46.606 | SUCCESS  | __main__:training_each_inductor_holdout:37 - svm - DPT_9 - test_f1_binary: 0.5520361990950226
# 2024-01-27 11:17:46.609 | SUCCESS  | __main__:training_each_inductor_holdout:39 - svm - DPT_9 - test_f1_macro: 0.7422757682591677
# 2024-01-27 11:17:46.610 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### xgboost INDUCTOR EXECUTING ####### - DPT_9


# 2024-01-27 11:17:46.612 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=None, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=None, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=None, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) - xgboost - DPT_9
# 2024-01-27 11:17:46.613 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'objective': ['reg:squarederror', 'binary:logistic', 'multi:softmax', 'binary:hinge'], 'n_estimators': [100, 1000], 'max_depth': [10, 30], 'learning_rate': [0.01, 0.5]} - xgboost - DPT_9
# 2024-01-27 11:17:46.614 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - xgboost - DPT_9
# 2024-01-27 22:41:37.617 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - xgboost - DPT_9
# 2024-01-27 22:41:37.621 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=0.5, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=10, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=100, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) 

# 2024-01-27 22:41:55.676 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS xgboost - DPT_9 #######


# 2024-01-27 22:41:55.682 | SUCCESS  | __main__:training_each_inductor_holdout:33 - xgboost - DPT_9 - test_precision: 0.5699481865284974
# 2024-01-27 22:41:55.686 | SUCCESS  | __main__:training_each_inductor_holdout:35 - xgboost - DPT_9 - test_recall: 0.4583333333333333
# 2024-01-27 22:41:55.689 | SUCCESS  | __main__:training_each_inductor_holdout:37 - xgboost - DPT_9 - test_f1_binary: 0.5080831408775982
# 2024-01-27 22:41:55.692 | SUCCESS  | __main__:training_each_inductor_holdout:39 - xgboost - DPT_9 - test_f1_macro: 0.7178540067282996
# 2024-01-27 22:41:55.693 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### nb INDUCTOR EXECUTING ####### - DPT_9


# 2024-01-27 22:41:55.695 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: MultinomialNB() - nb - DPT_9
# 2024-01-27 22:41:55.695 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05], 'force_alpha': [True, False], 'fit_prior': [True, False]} - nb - DPT_9
# 2024-01-27 22:41:55.696 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - nb - DPT_9
# 2024-01-27 22:41:58.144 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - nb - DPT_9
# 2024-01-27 22:41:58.147 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: MultinomialNB(alpha=0.0001, fit_prior=False, force_alpha=True) 

# 2024-01-27 22:41:58.178 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS nb - DPT_9 #######


# 2024-01-27 22:41:58.182 | SUCCESS  | __main__:training_each_inductor_holdout:33 - nb - DPT_9 - test_precision: 0.40638297872340423
# 2024-01-27 22:41:58.186 | SUCCESS  | __main__:training_each_inductor_holdout:35 - nb - DPT_9 - test_recall: 0.7958333333333333
# 2024-01-27 22:41:58.190 | SUCCESS  | __main__:training_each_inductor_holdout:37 - nb - DPT_9 - test_f1_binary: 0.5380281690140845
# 2024-01-27 22:41:58.193 | SUCCESS  | __main__:training_each_inductor_holdout:39 - nb - DPT_9 - test_f1_macro: 0.7074987056623311
# 2024-01-27 22:41:58.194 | SUCCESS  | __main__:<module>:72 - 

# [+][+][+] SUCCESS! [+][+][+]


# 2024-01-27 22:41:58.195 | INFO     | __main__:<module>:2 - 

# ####### DPT_5 EXECUTING #######


# 2024-01-27 22:41:58.199 | INFO     | __main__:<module>:14 - total_relevant_documents_of_the_board: 534 - DPT_5
# 2024-01-27 22:41:58.210 | INFO     | __main__:<module>:15 - Total irrelevant documents of the department: 2594
# 2024-01-27 22:41:58.213 | INFO     | __main__:<module>:18 - data_relevant_board: (534, 18) - DPT_5
# 2024-01-27 22:41:58.218 | INFO     | __main__:<module>:39 - data_irrelevant_board: (2594, 18) - DPT_5
# 2024-01-27 22:41:58.240 | INFO     | __main__:<module>:47 - df_final_dataset: (3128, 18) - DPT_5
# 2024-01-27 22:41:58.241 | DEBUG    | __main__:<module>:50 - [+] Start cleaning text and title
# 2024-01-27 22:42:23.282 | DEBUG    | __main__:<module>:55 - [+] End cleaning text and title
# 2024-01-27 22:42:23.288 | DEBUG    | __main__:<module>:61 - [+] Start TF-IDF
# 2024-01-27 22:42:26.192 | DEBUG    | __main__:<module>:65 - [+] End TF-IDF
# 2024-01-27 22:42:26.194 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### random_forest INDUCTOR EXECUTING ####### - DPT_5


# 2024-01-27 22:42:26.195 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: RandomForestClassifier(n_jobs=38, random_state=42) - random_forest - DPT_5
# 2024-01-27 22:42:26.196 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'n_estimators': [100, 400, 1000], 'max_depth': [10, 30, 100], 'criterion': ['entropy', 'log_loss', 'gini']} - random_forest - DPT_5
# 2024-01-27 22:42:26.196 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - random_forest - DPT_5
# 2024-01-27 22:45:05.618 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - random_forest - DPT_5
# 2024-01-27 22:45:05.620 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: RandomForestClassifier(max_depth=100, n_jobs=38, random_state=42) 

# 2024-01-27 22:45:06.143 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS random_forest - DPT_5 #######


# 2024-01-27 22:45:06.149 | SUCCESS  | __main__:training_each_inductor_holdout:33 - random_forest - DPT_5 - test_precision: 0.810126582278481
# 2024-01-27 22:45:06.153 | SUCCESS  | __main__:training_each_inductor_holdout:35 - random_forest - DPT_5 - test_recall: 0.29906542056074764
# 2024-01-27 22:45:06.157 | SUCCESS  | __main__:training_each_inductor_holdout:37 - random_forest - DPT_5 - test_f1_binary: 0.43686006825938567
# 2024-01-27 22:45:06.160 | SUCCESS  | __main__:training_each_inductor_holdout:39 - random_forest - DPT_5 - test_f1_macro: 0.681116601293872
# 2024-01-27 22:45:06.160 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### svm INDUCTOR EXECUTING ####### - DPT_5


# 2024-01-27 22:45:06.161 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: SVC(random_state=42) - svm - DPT_5
# 2024-01-27 22:45:06.162 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'C': [0.025, 0.08, 0.1, 0.5, 0.8, 1.0, 2.0, 10.0, 100.0, 500.0, 1000.0], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} - svm - DPT_5
# 2024-01-27 22:45:06.163 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - svm - DPT_5
# 2024-01-27 22:48:40.812 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - svm - DPT_5
# 2024-01-27 22:48:40.814 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: SVC(kernel='linear', random_state=42) 

# 2024-01-27 22:48:48.357 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS svm - DPT_5 #######


# 2024-01-27 22:48:48.362 | SUCCESS  | __main__:training_each_inductor_holdout:33 - svm - DPT_5 - test_precision: 0.7685185185185185
# 2024-01-27 22:48:48.365 | SUCCESS  | __main__:training_each_inductor_holdout:35 - svm - DPT_5 - test_recall: 0.3878504672897196
# 2024-01-27 22:48:48.368 | SUCCESS  | __main__:training_each_inductor_holdout:37 - svm - DPT_5 - test_f1_binary: 0.515527950310559
# 2024-01-27 22:48:48.370 | SUCCESS  | __main__:training_each_inductor_holdout:39 - svm - DPT_5 - test_f1_macro: 0.7220169540737029
# 2024-01-27 22:48:48.371 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### xgboost INDUCTOR EXECUTING ####### - DPT_5


# 2024-01-27 22:48:48.373 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=None, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=None, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=None, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) - xgboost - DPT_5
# 2024-01-27 22:48:48.374 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'objective': ['reg:squarederror', 'binary:logistic', 'multi:softmax', 'binary:hinge'], 'n_estimators': [100, 1000], 'max_depth': [10, 30], 'learning_rate': [0.01, 0.5]} - xgboost - DPT_5
# 2024-01-27 22:48:48.374 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - xgboost - DPT_5
# 2024-01-28 05:20:16.613 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - xgboost - DPT_5
# 2024-01-28 05:20:16.617 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=0.5, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=10, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=100, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) 

# 2024-01-28 05:20:29.114 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS xgboost - DPT_5 #######


# 2024-01-28 05:20:29.119 | SUCCESS  | __main__:training_each_inductor_holdout:33 - xgboost - DPT_5 - test_precision: 0.6423357664233577
# 2024-01-28 05:20:29.122 | SUCCESS  | __main__:training_each_inductor_holdout:35 - xgboost - DPT_5 - test_recall: 0.411214953271028
# 2024-01-28 05:20:29.125 | SUCCESS  | __main__:training_each_inductor_holdout:37 - xgboost - DPT_5 - test_f1_binary: 0.5014245014245013
# 2024-01-28 05:20:29.128 | SUCCESS  | __main__:training_each_inductor_holdout:39 - xgboost - DPT_5 - test_f1_macro: 0.7100712846184282
# 2024-01-28 05:20:29.129 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### nb INDUCTOR EXECUTING ####### - DPT_5


# 2024-01-28 05:20:29.130 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: MultinomialNB() - nb - DPT_5
# 2024-01-28 05:20:29.130 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05], 'force_alpha': [True, False], 'fit_prior': [True, False]} - nb - DPT_5
# 2024-01-28 05:20:29.131 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - nb - DPT_5
# 2024-01-28 05:20:31.589 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - nb - DPT_5
# 2024-01-28 05:20:31.591 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: MultinomialNB(alpha=0.01, force_alpha=True) 

# 2024-01-28 05:20:31.622 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS nb - DPT_5 #######


# 2024-01-28 05:20:31.627 | SUCCESS  | __main__:training_each_inductor_holdout:33 - nb - DPT_5 - test_precision: 0.5367965367965368
# 2024-01-28 05:20:31.630 | SUCCESS  | __main__:training_each_inductor_holdout:35 - nb - DPT_5 - test_recall: 0.5794392523364486
# 2024-01-28 05:20:31.634 | SUCCESS  | __main__:training_each_inductor_holdout:37 - nb - DPT_5 - test_f1_binary: 0.5573033707865169
# 2024-01-28 05:20:31.637 | SUCCESS  | __main__:training_each_inductor_holdout:39 - nb - DPT_5 - test_f1_macro: 0.7308129287152594
# 2024-01-28 05:20:31.638 | SUCCESS  | __main__:<module>:72 - 

# [+][+][+] SUCCESS! [+][+][+]


# 2024-01-28 05:20:31.639 | INFO     | __main__:<module>:2 - 

# ####### DPT_16 EXECUTING #######


# 2024-01-28 05:20:31.643 | INFO     | __main__:<module>:14 - total_relevant_documents_of_the_board: 439 - DPT_16
# 2024-01-28 05:20:31.652 | INFO     | __main__:<module>:15 - Total irrelevant documents of the department: 1558
# 2024-01-28 05:20:31.655 | INFO     | __main__:<module>:18 - data_relevant_board: (439, 18) - DPT_16
# 2024-01-28 05:20:31.660 | INFO     | __main__:<module>:39 - data_irrelevant_board: (1558, 18) - DPT_16
# 2024-01-28 05:20:31.678 | INFO     | __main__:<module>:47 - df_final_dataset: (1997, 18) - DPT_16
# 2024-01-28 05:20:31.678 | DEBUG    | __main__:<module>:50 - [+] Start cleaning text and title
# 2024-01-28 05:20:56.880 | DEBUG    | __main__:<module>:55 - [+] End cleaning text and title
# 2024-01-28 05:20:56.887 | DEBUG    | __main__:<module>:61 - [+] Start TF-IDF
# 2024-01-28 05:20:59.984 | DEBUG    | __main__:<module>:65 - [+] End TF-IDF
# 2024-01-28 05:20:59.986 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### random_forest INDUCTOR EXECUTING ####### - DPT_16


# 2024-01-28 05:20:59.987 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: RandomForestClassifier(n_jobs=38, random_state=42) - random_forest - DPT_16
# 2024-01-28 05:20:59.989 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'n_estimators': [100, 400, 1000], 'max_depth': [10, 30, 100], 'criterion': ['entropy', 'log_loss', 'gini']} - random_forest - DPT_16
# 2024-01-28 05:20:59.990 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - random_forest - DPT_16
# 2024-01-28 05:26:16.963 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - random_forest - DPT_16
# 2024-01-28 05:26:16.966 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: RandomForestClassifier(criterion='entropy', max_depth=100, n_jobs=38,
#                        random_state=42) 

# 2024-01-28 05:26:17.539 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS random_forest - DPT_16 #######


# 2024-01-28 05:26:17.547 | SUCCESS  | __main__:training_each_inductor_holdout:33 - random_forest - DPT_16 - test_precision: 0.6804123711340206
# 2024-01-28 05:26:17.550 | SUCCESS  | __main__:training_each_inductor_holdout:35 - random_forest - DPT_16 - test_recall: 0.375
# 2024-01-28 05:26:17.553 | SUCCESS  | __main__:training_each_inductor_holdout:37 - random_forest - DPT_16 - test_f1_binary: 0.48351648351648346
# 2024-01-28 05:26:17.556 | SUCCESS  | __main__:training_each_inductor_holdout:39 - random_forest - DPT_16 - test_f1_macro: 0.6885506945884304
# 2024-01-28 05:26:17.557 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### svm INDUCTOR EXECUTING ####### - DPT_16


# 2024-01-28 05:26:17.558 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: SVC(random_state=42) - svm - DPT_16
# 2024-01-28 05:26:17.559 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'C': [0.025, 0.08, 0.1, 0.5, 0.8, 1.0, 2.0, 10.0, 100.0, 500.0, 1000.0], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} - svm - DPT_16
# 2024-01-28 05:26:17.559 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - svm - DPT_16
# 2024-01-28 05:27:48.162 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - svm - DPT_16
# 2024-01-28 05:27:48.164 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: SVC(C=10.0, kernel='sigmoid', random_state=42) 

# 2024-01-28 05:27:51.162 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS svm - DPT_16 #######


# 2024-01-28 05:27:51.166 | SUCCESS  | __main__:training_each_inductor_holdout:33 - svm - DPT_16 - test_precision: 0.6692913385826772
# 2024-01-28 05:27:51.169 | SUCCESS  | __main__:training_each_inductor_holdout:35 - svm - DPT_16 - test_recall: 0.48295454545454547
# 2024-01-28 05:27:51.172 | SUCCESS  | __main__:training_each_inductor_holdout:37 - svm - DPT_16 - test_f1_binary: 0.5610561056105611
# 2024-01-28 05:27:51.175 | SUCCESS  | __main__:training_each_inductor_holdout:39 - svm - DPT_16 - test_f1_macro: 0.7291767014539292
# 2024-01-28 05:27:51.176 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### xgboost INDUCTOR EXECUTING ####### - DPT_16


# 2024-01-28 05:27:51.177 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=None, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=None, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=None, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) - xgboost - DPT_16
# 2024-01-28 05:27:51.178 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'objective': ['reg:squarederror', 'binary:logistic', 'multi:softmax', 'binary:hinge'], 'n_estimators': [100, 1000], 'max_depth': [10, 30], 'learning_rate': [0.01, 0.5]} - xgboost - DPT_16
# 2024-01-28 05:27:51.179 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - xgboost - DPT_16
# 2024-01-28 14:11:02.057 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - xgboost - DPT_16
# 2024-01-28 14:11:02.061 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: XGBClassifier(base_score=None, booster=None, callbacks=None,
#               colsample_bylevel=None, colsample_bynode=None,
#               colsample_bytree=None, device=None, early_stopping_rounds=None,
#               enable_categorical=False, eval_metric=None, feature_types=None,
#               gamma=None, grow_policy=None, importance_type=None,
#               interaction_constraints=None, learning_rate=0.5, max_bin=None,
#               max_cat_threshold=None, max_cat_to_onehot=None,
#               max_delta_step=None, max_depth=10, max_leaves=None,
#               min_child_weight=None, missing=nan, monotone_constraints=None,
#               multi_strategy=None, n_estimators=100, n_jobs=38, num_class=2,
#               num_parallel_tree=None, ...) 

# 2024-01-28 14:11:10.615 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS xgboost - DPT_16 #######


# 2024-01-28 14:11:10.622 | SUCCESS  | __main__:training_each_inductor_holdout:33 - xgboost - DPT_16 - test_precision: 0.6610169491525424
# 2024-01-28 14:11:10.626 | SUCCESS  | __main__:training_each_inductor_holdout:35 - xgboost - DPT_16 - test_recall: 0.4431818181818182
# 2024-01-28 14:11:10.630 | SUCCESS  | __main__:training_each_inductor_holdout:37 - xgboost - DPT_16 - test_f1_binary: 0.5306122448979592
# 2024-01-28 14:11:10.633 | SUCCESS  | __main__:training_each_inductor_holdout:39 - xgboost - DPT_16 - test_f1_macro: 0.7123920120195317
# 2024-01-28 14:11:10.634 | INFO     | __main__:training_each_inductor_holdout:6 - 

# ####### nb INDUCTOR EXECUTING ####### - DPT_16


# 2024-01-28 14:11:10.634 | DEBUG    | __main__:training_each_inductor_holdout:8 - [+] classifiers[key]['estimator']: MultinomialNB() - nb - DPT_16
# 2024-01-28 14:11:10.635 | DEBUG    | __main__:training_each_inductor_holdout:9 - [+] classifiers[key]['parameters']: {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05], 'force_alpha': [True, False], 'fit_prior': [True, False]} - nb - DPT_16
# 2024-01-28 14:11:10.636 | DEBUG    | __main__:training_each_inductor_holdout:19 - [+] Start GridSearchCV - nb - DPT_16
# 2024-01-28 14:11:13.102 | DEBUG    | __main__:training_each_inductor_holdout:21 - [+] End GridSearchCV - nb - DPT_16
# 2024-01-28 14:11:13.104 | INFO     | __main__:training_each_inductor_holdout:23 - search.best_estimator_: MultinomialNB(alpha=0.01, fit_prior=False, force_alpha=True) 

# 2024-01-28 14:11:13.128 | SUCCESS  | __main__:training_each_inductor_holdout:31 - 

# ####### METRICS nb - DPT_16 #######


# 2024-01-28 14:11:13.132 | SUCCESS  | __main__:training_each_inductor_holdout:33 - nb - DPT_16 - test_precision: 0.5234375
# 2024-01-28 14:11:13.135 | SUCCESS  | __main__:training_each_inductor_holdout:35 - nb - DPT_16 - test_recall: 0.7613636363636364
# 2024-01-28 14:11:13.139 | SUCCESS  | __main__:training_each_inductor_holdout:37 - nb - DPT_16 - test_f1_binary: 0.6203703703703703
# 2024-01-28 14:11:13.142 | SUCCESS  | __main__:training_each_inductor_holdout:39 - nb - DPT_16 - test_f1_macro: 0.7398592846706054
# 2024-01-28 14:11:13.143 | SUCCESS  | __main__:<module>:72 - 

# [+][+][+] SUCCESS! [+][+][+]


